In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn import preprocessing
import random

### Importing OTU table, normalization (change to relative abundances), set training and testing groups

In [51]:
# Import data into a pandas dataframe
otu_table = pd.read_csv('EGAD00001003453.otu_table.99.denovo', sep='\t', header=0, index_col=0)

#Transpose OTU table so that samples are rows and OTUs are columns
otu_table = otu_table.T

#Convert to relative abundances
#Normalize each person to 1 (change to relative abundances)
otu_table_normalized = otu_table.div(otu_table.sum(axis=1), axis=0)
print(otu_table_normalized)

OTU_ID  denovo7709  denovo5395  denovo11322  denovo44859  denovo44858  \
G37960    0.000000         0.0          0.0          0.0          0.0   
G37961    0.000000         0.0          0.0          0.0          0.0   
G37962    0.000000         0.0          0.0          0.0          0.0   
G37963    0.000000         0.0          0.0          0.0          0.0   
G37964    0.000000         0.0          0.0          0.0          0.0   
G37965    0.000000         0.0          0.0          0.0          0.0   
G37966    0.000000         0.0          0.0          0.0          0.0   
G37967    0.000000         0.0          0.0          0.0          0.0   
G37968    0.000000         0.0          0.0          0.0          0.0   
G37969    0.000000         0.0          0.0          0.0          0.0   
G37970    0.000000         0.0          0.0          0.0          0.0   
G37971    0.000000         0.0          0.0          0.0          0.0   
G37972    0.000000         0.0          0.0        

In [52]:
#Make training and testing sets (50/50 for now)

#Scramble existing table with hard-coded random seed
otu_table_normalized = otu_table_normalized.sample(frac=1, random_state=1)

#Make first half training and second half testing
total_people = otu_table_normalized.shape[0]
print(total_people)
midway = total_people/2
training_table = otu_table_normalized.iloc[0:int(midway)]
testing_table = otu_table_normalized.iloc[int(midway):]


1010


### Set conditions to test:

In [53]:
N = [5, 10, 50, 100, 250]
weight_setup = ['even', 'uneven_100x', 'uneven_10x', 'uneven_graduated']#, 'uneven_bins']
num_bins = 3

#Number of times to run each condition (N and weight setup)
num_repeats = 5

### Functions

In [54]:
def calculate_weights(n,weight):
    if weight == 'even':
        return [1/n]*n
    elif weight == 'uneven_100x':
        x = 1/(100 + (n-1))
        return [100*x] + [x]*(n-1)
    elif weight == 'uneven_10x':
        x = 1/(10 + (n-1))
        return [10*x] + [x]*(n-1)
    elif weight == 'uneven_graduated':
        decreasing_list = list(range(n,0,-1))
        return [x/sum(decreasing_list) for x in decreasing_list]
    #elif weight == 'uneven_bins':
        #n/

    

### Master loop

In [94]:
state = [[0]*len(N) for i in range(num_repeats)]
df_list = []

for i in range(num_repeats):
    for j, n in enumerate(N):
        #Keep track of random seeds used to generate N random samples from OTU table
        state[i][j] = random.getstate()
        #Generate random sample of size n
        random_sample = training_table.sample(n=n, replace=True)
        
        for m, weight in enumerate(weight_setup):
            #Set up actual weights
            weight_list = calculate_weights(n,weight)
            
            #Calculate weighted OTU table
            #weighted_table = [random_sample.iloc[i]*weight_list[i] for i in range(n)]
            for k in range(n):
                random_sample.iloc[k] = random_sample.iloc[k]*weight_list[k]
                
            #Calculate weighted mean
            indiv_series = random_sample.mean(axis=0)
            
            #Store data + info/metadata as pandas Series
            sample_name = str(n) + "_" + weight + "_" + str(i)
            sample_number = i*len(N)*len(weight_setup) + j*len(weight_setup) + m
            indiv_series.loc['sample_name'] = sample_name
            indiv_series.loc['N'] = n
            indiv_series.loc['weight_setup'] = weight
            indiv_series.loc['weight_vector'] = weight_list
            indiv_series.loc['random_seed'] = random.getstate()
            
            
            indiv_df = pd.DataFrame(indiv_series, columns=[str(sample_number)])
            #print(indiv_df)
            df_list.append(indiv_df)
            
final_matrix = pd.concat(df_list, axis=1)
print(final_matrix.tail())

                                                               0  \
OTU_ID                                                             
sample_name                                             5_even_0   
N                                                              5   
weight_setup                                                even   
weight_vector                          [0.2, 0.2, 0.2, 0.2, 0.2]   
random_seed    (3, (2147483648, 2561330682, 2716738939, 30638...   

                                                               1  \
OTU_ID                                                             
sample_name                                      5_uneven_100x_0   
N                                                              5   
weight_setup                                         uneven_100x   
weight_vector  [0.9615384615384616, 0.009615384615384616, 0.0...   
random_seed    (3, (2147483648, 2561330682, 2716738939, 30638...   

                                              

In [95]:
final_matrix

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
OTU_ID,,,,,,,,,,,,,,,,,,,,,
denovo7709,3.62437e-07,3.48497e-09,2.48927e-10,3.31902e-11,4.15679e-07,3.81357e-09,2.00714e-10,3.64935e-12,3.08704e-09,2.07184e-11,...,1.18034e-11,3.51846e-13,2.27892e-08,1.14519e-10,1.05063e-12,1.03882e-14,6.12186e-09,1.75411e-11,6.77264e-14,2.95136e-16
denovo5395,7.33541e-07,7.05328e-09,5.03806e-10,1.00761e-10,2.7712e-07,2.54238e-09,1.3381e-10,2.4329e-12,1.23272e-07,7.71583e-08,...,2.3933e-11,7.26593e-13,1.82472e-08,9.16946e-11,8.41235e-13,1.14433e-14,3.49048e-08,1.00014e-10,3.86153e-13,1.76953e-15
denovo11322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
denovo44859,0,0,0,0,0,0,0,0,0,0,...,0,0,2.89469e-09,1.45462e-11,1.33451e-13,1.74412e-15,7.29309e-10,1.56351e-10,6.01826e-12,4.79419e-14
denovo44858,0,0,0,0,0,0,0,0,0,0,...,1.2557e-12,2.9546e-14,4.41702e-10,2.21961e-12,2.03634e-14,3.9517e-16,1.68687e-09,4.83344e-12,1.86619e-14,1.03223e-16
denovo41472,0,0,0,0,0,0,0,0,0,0,...,1.892e-12,5.05199e-14,1.09638e-09,5.50946e-12,5.05455e-14,9.00811e-16,5.19131e-10,1.48748e-12,5.74317e-15,2.74659e-17
denovo41473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3.89698e-10,1.11661e-12,4.31125e-15,1.23855e-18
denovo41470,0,0,0,0,0,0,0,0,8.35516e-09,5.60749e-11,...,0,0,1.02521e-09,5.15181e-12,4.72643e-14,8.9849e-16,3.7577e-10,1.0767e-12,4.15716e-15,2.57326e-18
denovo41471,0,0,0,0,0,0,0,0,5.04745e-09,3.38755e-11,...,0,0,0,0,0,0,4.22269e-10,1.20994e-12,4.67158e-15,1.66439e-17


In [96]:
final_matrix.to_csv('final_matrix.csv')